# OpenAI Price Calculator

[OpenAI API](https://platform.openai.com/) の価格を計算するための Notebook です。正確なトークン数を得るために、公式の Tokenizer である [tiktoken](https://github.com/openai/tiktoken) を使用します。 API の単価は変わるため、 [Pricing](https://openai.com/pricing) で最新の単価をご確認ください。本ノートブックの計算結果は入力値に依存するため、自らの責任でもってご利用ください。

Notebook を動かす前に、 OpenAI API の API KEY と Organization ID の確認が必要です。いずれも Manage Account のメニューから確認できます。取得した API KEY と Organization ID は `os.environ` などを使い事前に設定しておいてください。これらは機微な情報のため、 GitHub などに誤ってコミットしないよう注意してください。

!pip install tiktoken

In [ ]:
import os
import json
import requests
import pandas as pd


JAQKET_TRAIN_DATASET = (
    "https://jaqket.s3.ap-northeast-1.amazonaws.com/data/aio_02/aio_02_train.jsonl"
)
JAQKET_DEV_DATASET = (
    "https://jaqket.s3.ap-northeast-1.amazonaws.com/data/aio_02/aio_02_dev_v1.0.jsonl"
)

In [ ]:
def read_jaqket_dataset(dataset_url: str) -> pd.DataFrame:
    file_name = os.path.basename(dataset_url)
    location = os.path.join(f"data/{file_name}")
    if not os.path.exists(location):
        response = requests.get(dataset_url)
        with open(location, mode="wb") as f:
            f.write(response.content)

    return pd.read_json(location, lines=True)

In [ ]:
df_train = read_jaqket_dataset(JAQKET_TRAIN_DATASET)
df_dev = read_jaqket_dataset(JAQKET_DEV_DATASET)

In [ ]:
df_train.question

In [ ]:
import tiktoken


def calculate_price(model_name: str, input_unit_price: float, output_unit_price: float,
                    quiz_df: pd.DataFrame) -> dict:
    tokenizer = tiktoken.encoding_for_model(model_name)
    num_input_tokens = quiz_df.question.map(lambda q: len(tokenizer.encode(q))).sum()
    num_output_tokens = quiz_df.answers.map(lambda a: len(tokenizer.encode(a[0]))).sum()
    input_price = num_input_tokens / 1000 * input_unit_price
    output_price = num_output_tokens / 1000 * output_unit_price
    result = {
        "model_name": model_name,
        "num_input_tokens": num_input_tokens,
        "num_output_tokens": num_output_tokens,
        "input_price": input_price,
        "output_price": output_price,
        "total": input_price + output_price
    }
    return result

In [ ]:
calculate_price("text-davinci-003", 0.0200, 0.0200, df_dev)

In [ ]:
calculate_price("gpt-3.5-turbo", 0.0015, 0.002, df_dev)

In [ ]:
calculate_price("curie", 0.003, 0.003, df_train)